## MACHNE LEARNING

## Task 3

Object Detection and Tracking:Create a model that can detect and track objects in real-time
video feeds, which can be applied to surveillance, autonomous vehicles, orrobotics.

In [ ]:
import cv2
import numpy as np
from sort import Sort

# Load YOLO model for object detection
yolo_net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")  # You'll need to download YOLO weights and configuration files
layer_names = yolo_net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in yolo_net.getUnconnectedOutLayers()]

# Initialize SORT (Simple Online and Realtime Tracking)
tracker = Sort()

# Initialize video capture
cap = cv2.VideoCapture("input_video.mp4")  # Replace with your video source

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Object detection
    height, width, _ = frame.shape
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    yolo_net.setInput(blob)
    outs = yolo_net.forward(output_layers)

    class_ids = []
    confidences = []
    boxes = []

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:  # Adjust confidence threshold as needed
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # Apply non-maximum suppression to remove overlapping boxes
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    # Object tracking
    if len(indexes) > 0:
        tracked_objects = tracker.update(np.array(boxes))

        for obj in tracked_objects:
            x, y, w, h, track_id = obj
            label = f"ID: {track_id}"
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    cv2.imshow("Object Detection and Tracking", frame)
    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()


**Report: Real-time Object Detection and Tracking Model**

**Objective:**
The objective of this project is to create a real-time object detection and tracking model that can be applied to various scenarios, such as surveillance, autonomous vehicles, or robotics.

**Implementation:**
1. **Dependencies:**
   - Python
   - OpenCV (Open Source Computer Vision Library)
   - SORT (Simple Online and Realtime Tracking) Algorithm
   - Pre-trained YOLO (You Only Look Once) Model

2. **Object Detection:**
   - We used a pre-trained YOLO model for object detection. YOLO is a deep learning model that can identify objects in images and videos efficiently.
   - We loaded the YOLO weights and configuration files to initialize the model.
   - OpenCV was used to capture video frames from a source, such as a webcam or video file.
   - For each frame, we applied YOLO for object detection.
   - The YOLO model provided us with class IDs, confidences, and bounding boxes for detected objects.
   - We filtered the detections based on a confidence threshold (adjustable) and retrieved the class IDs and bounding box coordinates.

3. **Non-maximum Suppression:**
   - We applied non-maximum suppression (NMS) to eliminate overlapping boxes. This process ensured that we only kept the most confident and non-overlapping detections.

4. **Object Tracking:**
   - We used the SORT (Simple Online and Realtime Tracking) algorithm to track objects over time.
   - SORT is a tracking algorithm that associates objects in consecutive frames and assigns unique IDs to each object for tracking.
   - For each detected object, SORT predicted its position and tracked its movement over time.
   - The algorithm handled cases where objects may temporarily disappear or reappear in the video feed.

5. **Visualization:**
   - We drew bounding boxes around the detected and tracked objects, with a unique ID for each tracked object displayed above the box.
   - The resulting frames with visualizations were displayed in real-time using OpenCV.

6. **User Interaction:**
   - We provided the option to exit the real-time video feed by pressing the 'Esc' key.

**Conclusion:**
This real-time object detection and tracking model successfully combines YOLO for object detection and the SORT algorithm for object tracking. It can be applied to a wide range of applications, including surveillance, autonomous vehicles, and robotics. The model is capable of detecting and tracking objects in real-time video feeds and providing unique identifiers for each tracked object. Users can further fine-tune the model's parameters, such as confidence thresholds, to adapt it to specific use cases.

**Future Improvements:**
- Optimization for real-time performance and speed.
- Integration with external sensors (e.g., lidar, radar) for improved tracking accuracy in autonomous vehicles.
- Deployment on edge devices for low-latency applications.
- Enhanced user interfaces for customization and real-time monitoring.